In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle 
import os

# !pip3 install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cpu.html

import torch_geometric
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool
from torch_geometric.utils import to_networkx

import requests

all_years = [f'20{str(i).zfill(2)}' for i in range(0, 22)]
url = 'https://api.lens.org/scholarly/search'

query = '%22Angewandte%20Chemie%22%20OR%20%22Blood%22%20OR%20%22Cancer%20Cell%22%20OR%20%22Cancer%20Discovery%22%20OR%20%22Cancer%20Research%22%20OR%20%22Cell%22%20OR%20%22Cell%20Host%20%26%20Microbe%22%20OR%20%22Cell%20Metabolism%22%20OR%20%22Cell%20Stem%20Cell%22%20OR%20%22Chemistry%20%26%20Biology%22%20OR%20%22The%20EMBO%20Journal%22%20OR%20%22Genes%20%26%20Development%22%20OR%20%22Immunity%22%20OR%20%22Journal%20of%20Neurology%22%20OR%20%22Journal%20of%20the%20American%20Chemical%20Society%22%20OR%20%22JAMA%22%20OR%20%22Journal%20of%20Biological%20Chemistry%22%20OR%20%22Journal%20of%20Cell%20Biology%22%20OR%20%22Journal%20of%20Clinical%20Investigation%22%20OR%20%22Journal%20of%20Experimental%20Medicine%22%20OR%20%22Journal%20of%20Medicinal%20Chemistry%22%20OR%20%22The%20Lancet%22%20OR%20%22Nature%20Cell%20Biology%22%20OR%20%22Nature%20Chemical%20Biology%22%20OR%20%22Nature%20Chemistry%22%20OR%20%22Nature%20Medicine%22%20OR%20%22Nature%20Methods%22%20OR%20%22Nature%22%20OR%20%22Nature%20Biotechnology%22%20OR%20%22The%20New%20England%20Journal%20of%20Medicine%22%20OR%20%22Neuron%22%20OR%20%22Nature%20Genetics%22%20OR%20%22Nature%20Immunology%22%20OR%20%22Nature%20Neuroscience%22%20OR%20%22Nature%20Structural%20%26%20Molecular%20Biology%22%20OR%20%22PLOS%20Biology%22%20OR%20%22PLOS%20Genetics%22%20OR%20%22PLOS%20Pathogens%22%20OR%20%22Proceedings%20of%20the%20National%20Academy%20of%20Sciences%20of%20the%20United%20States%20of%20America%22%20OR%20%22Science%20Signaling%22%20OR%20%22Science%20Translational%20Medicine%22%20OR%20%22Science%22)%20OR%20'
query = query.replace('%20', ' ')
query = query.replace('%22', '')
query = query.split('OR')
query = [x.strip() for x in query]
print(query)
data = '''
            "query": {
                "match_phrase":{
                        "source.title.exact": ['']
                }
            },
            "size": 1,
            "sort": [
                {
                        "year_published": "desc"
                }
            ]
        }
'''
headers = {'Authorization': 'Bearer your-access-token', 'Content-Type': 'application/json'}
response = requests.post(url, data=data, headers=headers)
if response.status_code != requests.codes.ok:
  print(response.status_code)
else:
  print(response.text)


/Users/devpatelio/miniconda3/envs/connexsci/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Angewandte Chemie', 'Blood', 'Cancer Cell', 'Cancer Discovery', 'Cancer Research', 'Cell', 'Cell Host %26 Microbe', 'Cell Metabolism', 'Cell Stem Cell', 'Chemistry %26 Biology', 'The EMBO Journal', 'Genes %26 Development', 'Immunity', 'Journal of Neurology', 'Journal of the American Chemical Society', 'JAMA', 'Journal of Biological Chemistry', 'Journal of Cell Biology', 'Journal of Clinical Investigation', 'Journal of Experimental Medicine', 'Journal of Medicinal Chemistry', 'The Lancet', 'Nature Cell Biology', 'Nature Chemical Biology', 'Nature Chemistry', 'Nature Medicine', 'Nature Methods', 'Nature', 'Nature Biotechnology', 'The New England Journal of Medicine', 'Neuron', 'Nature Genetics', 'Nature Immunology', 'Nature Neuroscience', 'Nature Structural %26 Molecular Biology', 'PLOS Biology', 'PLOS Genetics', 'PLOS Pathogens', 'Proceedings of the National Academy of Sciences of the United States of America', 'Science Signaling', 'Science Translational Medicine', 'Science)', '']
401

In [2]:
import tqdm
import json 
import pandas as pd

FILE = 'ml/gnn/arxiv-metadata-oai-snapshot.json'
LENS = 'ml/data/export-dd.json'
ROOT = os.getcwd().replace('ml/gnn', '')

metadata = []
train_size = 100000

with open(ROOT+FILE, 'r') as file:
         for line in file:
            metadata.append(json.loads(line))
            train_size -= 1
            
            if train_size == 0: break
            
df = pd.DataFrame(metadata)
db = pd.read_json(ROOT+LENS)

df.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [3]:
db.head()
# entry = db['funding'][2]
# print(entry)
# # remove_dict_warp = [entry[x]['issn'] for x in range(len(entry))]

,abstract,authors,chemicals,date_published,end_page,fields_of_study,funding,issue,keywords,lens_id,...,patent_citations_count,publication_type,references,scholarly_citations_count,source,source_urls,start_page,title,volume,year_published
0,"NLGN3 links oxytocin signaling, protein synthe...","[{'affiliations': [{'addresses': [], 'ids': []...",[],{'date': '2020-08-18T00:00:00+00:00'},None,"[{'name': 'Psychology'}, {'name': 'Prosocial b...",[],645,[],000-000-326-262-378,...,0,journal article,[{'lens_id': '077-677-839-990-811'}],0,"{'country': 'United States', 'issn': [{'type':...","[{'type': 'unknown', 'url': 'https://stke.scie...",None,A prosocial neuroligin,13,2020
1,Summary Females of many animal species behave ...,[{'affiliations': [{'addresses': [{'country_co...,"[{'mesh_id': 'D029721', 'registry_number': '0'...",{'date': '2009-02-26T00:00:00+00:00'},518,"[{'name': 'Internal medicine'}, {'name': 'Endo...",[],4,[],000-004-303-413-936,...,1,journal article,"[{'lens_id': '001-640-611-550-282'}, {'lens_id...",223,"{'country': 'United States', 'issn': [{'type':...","[{'type': 'html', 'url': 'https://www.ncbi.nlm...",511,Sensory neurons in the Drosophila genital trac...,61,2009
2,The enormous complexity of the central nervous...,[{'affiliations': [{'addresses': [{'country_co...,[],{'date': '2017-11-01T00:00:00+00:00'},571,"[{'name': 'Posttranslational modification'}, {...","[{'country': None, 'funding_id': None, 'org': ...",3,"[{'value': 'brain'}, {'value': 'central nervou...",000-004-360-715-324,...,0,journal article,"[{'lens_id': '000-005-937-882-552'}, {'lens_id...",50,"{'country': 'United States', 'issn': [{'type':...","[{'type': 'html', 'url': 'https://www.scienced...",558,A Primer on Concepts and Applications of Prote...,96,2017
3,T cell receptor (TCR) signaling is initiated a...,[{'affiliations': [{'addresses': [{'country_co...,"[{'mesh_id': 'D000199', 'registry_number': '0'...",{'date': None},127,"[{'name': 'Actin'}, {'name': 'Membrane protein...","[{'country': 'United States', 'funding_id': 'P...",1,[],000-009-038-775-698,...,7,journal article,"[{'lens_id': '000-290-175-402-908'}, {'lens_id...",767,"{'country': 'United States', 'issn': [{'type':...","[{'type': None, 'url': 'http://www.cell.com/im...",117,T cell receptor-proximal signals are sustained...,25,2006
4,None,[],[],{'date': '2001-03-27T00:00:00+00:00'},tw15,"[{'name': 'Blocking (statistics)'}, {'name': '...",[],75,[],000-009-723-468-418,...,0,None,[],0,"{'country': 'United States', 'issn': [{'type':...",[],tw15,Blocking a Survival Pathway,2001,2001


In [4]:
entry = db['references'][1]
print(entry)

# remove_dict_warp = [entry[x]['lens_id'] for x in range(len(entry))]
# print(remove_dict_warp)

[{'lens_id': '001-640-611-550-282'}, {'lens_id': '002-458-966-536-16X'}, {'lens_id': '002-803-224-613-87X'}, {'lens_id': '003-418-993-901-126'}, {'lens_id': '004-438-236-660-50X'}, {'lens_id': '005-202-691-673-156'}, {'lens_id': '005-369-043-101-20X'}, {'lens_id': '006-450-833-252-496'}, {'lens_id': '006-534-343-781-123'}, {'lens_id': '015-559-212-048-834'}, {'lens_id': '015-762-010-366-010'}, {'lens_id': '016-900-611-907-811'}, {'lens_id': '017-541-568-934-906'}, {'lens_id': '017-599-816-378-171'}, {'lens_id': '019-302-563-953-854'}, {'lens_id': '020-253-000-313-834'}, {'lens_id': '028-639-801-854-30X'}, {'lens_id': '030-551-071-061-230'}, {'lens_id': '034-177-009-965-114'}, {'lens_id': '035-110-032-606-208'}, {'lens_id': '036-363-253-740-509'}, {'lens_id': '038-134-437-009-706'}, {'lens_id': '040-888-893-005-867'}, {'lens_id': '043-863-871-696-108'}, {'lens_id': '051-366-252-097-918'}, {'lens_id': '053-335-250-805-061'}, {'lens_id': '053-476-024-525-984'}, {'lens_id': '056-724-362-07

In [5]:
def get_unique(data, column):
    all_categories = []
    for y, entry in enumerate(data[column]):
        entry_categories = [data[column].iloc[y][x]['name'] for x in range(len(data[column].iloc[y]))]
        all_categories += list(set(entry_categories))
    
    return list(set(all_categories))

def format_list_dict(data, column, name):
    base_entry = []
    for y, entry in enumerate(data[column]):
        try:
            remove_dict_warp = [entry[x][name] for x in range(len(entry))]
            base_entry.append(remove_dict_warp)
        except:
            base_entry.append(entry)
    return base_entry


def get_author_list(line):
    return [e[1] + ' ' + e[0] for e in line]


def get_category_list(line):
    return list(line.split(" "))


df['cleaned_authors_list'] = df['authors_parsed'].map(get_author_list)
df['category_list'] = df['categories'].map(get_category_list)
df = df.drop(['submitter', 'authors', 
             'comments', 'journal-ref', 
             'doi', 'report-no', 'license', 
             'versions', 'update_date', 
             'abstract', 'authors_parsed', 
             'categories'], axis=1)

fields_of_study = get_unique(db, 'fields_of_study')
db['fields_of_study'] = format_list_dict(db, 'fields_of_study', 'name')
db['references'] = format_list_dict(db, 'references', 'lens_id')
db['funding'] = format_list_dict(db, 'funding', 'country')
db['keywords'] = format_list_dict(db, 'keywords', 'value')

In [6]:
print(len(fields_of_study))

24183


In [7]:
cd = db.drop(['authors', 'end_page', 'date_published', 'issue', 'funding', 'patent_citations_count', 'scholarly_citations_count', 'publication_type', 'source', 'source_urls', 'start_page', 'volume', 'year_published'], axis=1)
print(len(cd))
cd.head()


50000


,abstract,chemicals,fields_of_study,keywords,lens_id,mesh_terms,references,title
0,"NLGN3 links oxytocin signaling, protein synthe...",[],"[Psychology, Prosocial behavior, Neuroscience,...",[],000-000-326-262-378,[],[077-677-839-990-811],A prosocial neuroligin
1,Summary Females of many animal species behave ...,"[{'mesh_id': 'D029721', 'registry_number': '0'...","[Internal medicine, Endocrinology, Receptor, f...",[],000-004-303-413-936,"[{'mesh_heading': 'Animals', 'mesh_id': 'D0008...","[001-640-611-550-282, 002-458-966-536-16X, 002...",Sensory neurons in the Drosophila genital trac...
2,The enormous complexity of the central nervous...,[],"[Posttranslational modification, Proteome, Bio...","[brain, central nervous system, cerebrospinal ...",000-004-360-715-324,"[{'mesh_heading': 'Animals', 'mesh_id': 'D0008...","[000-005-937-882-552, 000-305-151-801-372, 000...",A Primer on Concepts and Applications of Prote...
3,T cell receptor (TCR) signaling is initiated a...,"[{'mesh_id': 'D000199', 'registry_number': '0'...","[Actin, Membrane protein, T-cell receptor, Cal...",[],000-009-038-775-698,"[{'mesh_heading': 'Actins', 'mesh_id': 'D00019...","[000-290-175-402-908, 001-725-607-297-096, 001...",T cell receptor-proximal signals are sustained...
4,None,[],"[Blocking (statistics), Cell biology, Biology,...",[],000-009-723-468-418,[],[],Blocking a Survival Pathway


In [8]:
from itertools import chain

def apply_mesh_id(data, column):
    all_mesh_ids = []
    for y, entry in enumerate(data[column]):
        ids = [entry[x]['mesh_id'] for x in range(len(entry))]
        all_mesh_ids.append(ids)
    return all_mesh_ids

cd['chemicals'], cd['mesh_terms'] = apply_mesh_id(data=cd, column='chemicals'), apply_mesh_id(data=cd, column='mesh_terms')
cd.head()


,abstract,chemicals,fields_of_study,keywords,lens_id,mesh_terms,references,title
0,"NLGN3 links oxytocin signaling, protein synthe...",[],"[Psychology, Prosocial behavior, Neuroscience,...",[],000-000-326-262-378,[],[077-677-839-990-811],A prosocial neuroligin
1,Summary Females of many animal species behave ...,"[D029721, D010455, D018000, C489174, C526291]","[Internal medicine, Endocrinology, Receptor, f...",[],000-004-303-413-936,"[D000818, D004330, D029721, D005260, D017952, ...","[001-640-611-550-282, 002-458-966-536-16X, 002...",Sensory neurons in the Drosophila genital trac...
2,The enormous complexity of the central nervous...,[],"[Posttranslational modification, Proteome, Bio...","[brain, central nervous system, cerebrospinal ...",000-004-360-715-324,"[D000818, D001921, D002853, D006801, D013058, ...","[000-005-937-882-552, 000-305-151-801-372, 000...",A Primer on Concepts and Applications of Prote...
3,T cell receptor (TCR) signaling is initiated a...,"[D000199, D000906, D006649, D011948, D051746, ...","[Actin, Membrane protein, T-cell receptor, Cal...",[],000-009-038-775-698,"[D000199, D000818, D000906, D002478, D006649, ...","[000-290-175-402-908, 001-725-607-297-096, 001...",T cell receptor-proximal signals are sustained...
4,None,[],"[Blocking (statistics), Cell biology, Biology,...",[],000-009-723-468-418,[],[],Blocking a Survival Pathway


In [9]:
#set up neo4j driver to instance
from neo4j import GraphDatabase
import time

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        print('response sent successfully')
        return response

conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="neo4j",              
                       pwd="iter-v0001")

df.head()

,id,title,cleaned_authors_list,category_list
0,0704.0001,Calculation of prompt diphoton production cros...,"[C. Balázs, E. L. Berger, P. M. Nadolsky, C. -...",[hep-ph]
1,0704.0002,Sparsity-certifying Graph Decompositions,"[Ileana Streinu, Louis Theran]","[math.CO, cs.CG]"
2,0704.0003,The evolution of the Earth-Moon system based o...,[Hongjun Pan],[physics.gen-ph]
3,0704.0004,A determinant of Stirling cycle numbers counts...,[David Callan],[math.CO]
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,"[Wael Abu-Shammala, Alberto Torchinsky]","[math.CA, math.FA]"


In [10]:
conn.query('CREATE CONSTRAINT categories IF NOT EXISTS ON (c:Category)      ASSERT c.category IS UNIQUE')
conn.query('CREATE CONSTRAINT papers IF NOT EXISTS ON (p:Paper)     ASSERT p.id IS UNIQUE')
conn.query('CREATE CONSTRAINT mesh_entity iF NOT EXISTS on (m:Mesh)  ASSERT m.mesh_id is UNIQUE')

response sent successfully
response sent successfully
response sent successfully


[]

In [11]:
# cd[cd['id']=='023-087-054-076-174']

In [12]:
from xml.dom.minidom import parseString


def insert_data(query, rows, batch_size = 10000):
    # Function to handle the updating the Neo4j database in batch mode.
    
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):
        res = conn.query(query, 
                        parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        try:
            total += res[0]['total']
        except:
            total += 0
        batch += 1
        result = {"total":total, 
                "batches":batch, 
                "time":time.time()-start}
        print(result)
    return result

def add_categories(categories):
    query = ''' 
            UNWIND $rows AS row
            MERGE (c:Category {category: row.category})
            RETURN count(*) as total
            '''
    return conn.query(query, parameters={'rows':categories.to_dict('records')})

def add_mesh_term(mesh_terms):
    query = '''
            UNWIND $rows AS row
            MERGE (m:Mesh {mesh_id: row.mesh_term})
            RETURN count(*) as total
            '''
    return conn.query(query, parameters={'rows':mesh_terms.to_dict('records')})

def add_papers(rows, batch_size=10000):
    query = '''

    UNWIND $rows as row
    MERGE (p:Paper {id:row.id}) ON CREATE SET p.title = row.title
    
    // connect categories
    WITH row, p //take first paper
    UNWIND row.category_list AS category_name //get all categories for that paper
    MATCH (c:Category {category: category_name}) //match category of paper with all possible categories
    MERGE (p)-[:IN_CATEGORY]->(c)   

    // connect mesh elements
    WITH row, p //take first mesh element
    UNWIND row.mesh_term AS paper_mesh_id //get all mesh ids for that paper
    MATCH (m:Mesh {mesh_id: paper_mesh_id}) //match mesh id of paper with all possible mesh elements
    MERGE (p)-[:HAS_MeSH]->(m)

    // connect references
    WITH distinct row, p
    UNWIND row.refs AS ref_id
    MATCH (g:Paper {id: ref_id})
    MERGE (p)-[:REFERENCES]->(g)

    

    RETURN count(*) as total

    '''

    return insert_data(query, rows, batch_size=batch_size)

In [13]:
cd.head()

,abstract,chemicals,fields_of_study,keywords,lens_id,mesh_terms,references,title
0,"NLGN3 links oxytocin signaling, protein synthe...",[],"[Psychology, Prosocial behavior, Neuroscience,...",[],000-000-326-262-378,[],[077-677-839-990-811],A prosocial neuroligin
1,Summary Females of many animal species behave ...,"[D029721, D010455, D018000, C489174, C526291]","[Internal medicine, Endocrinology, Receptor, f...",[],000-004-303-413-936,"[D000818, D004330, D029721, D005260, D017952, ...","[001-640-611-550-282, 002-458-966-536-16X, 002...",Sensory neurons in the Drosophila genital trac...
2,The enormous complexity of the central nervous...,[],"[Posttranslational modification, Proteome, Bio...","[brain, central nervous system, cerebrospinal ...",000-004-360-715-324,"[D000818, D001921, D002853, D006801, D013058, ...","[000-005-937-882-552, 000-305-151-801-372, 000...",A Primer on Concepts and Applications of Prote...
3,T cell receptor (TCR) signaling is initiated a...,"[D000199, D000906, D006649, D011948, D051746, ...","[Actin, Membrane protein, T-cell receptor, Cal...",[],000-009-038-775-698,"[D000199, D000818, D000906, D002478, D006649, ...","[000-290-175-402-908, 001-725-607-297-096, 001...",T cell receptor-proximal signals are sustained...
4,None,[],"[Blocking (statistics), Cell biology, Biology,...",[],000-009-723-468-418,[],[],Blocking a Survival Pathway


In [14]:
# fields_of_study
import numpy as np
cd = cd.drop(['abstract', 'chemicals', 'keywords'], axis=1)

categories = pd.DataFrame(columns=['category'], data = fields_of_study).drop_duplicates(subset=['category'])
categories.head()

unique_meshes = list(set(list(chain.from_iterable(list(cd['mesh_terms'])))))

mesh_terms = pd.DataFrame(columns=['mesh_term'], data=unique_meshes).drop_duplicates(subset=['mesh_term'])
print(len(unique_meshes))

cd.head()

cd.columns = ['category_list', 'id', 'mesh_term', 'refs', 'title']
remove_entries = [i for i, x in enumerate(list(cd['category_list'])) if x==[]]

if len(remove_entries) != 0:
    cd = cd.drop([cd.index[x] for x in remove_entries], axis=0)

ref_check = []
for i, x in enumerate(list(cd['refs'])):
    if x != []:
        ref_check.append(x)
    else:
        ref_check.append([])

cd['refs'] = ref_check

# print(len(cd))


add_categories(categories)
add_mesh_term(mesh_terms)

add_papers(cd)

# # # conn.query('''MATCH (n) 
# # #               RETURN n LIMIT 300
# # #            ''')

8319
response sent successfully
response sent successfully
response sent successfully
{'total': 4196, 'batches': 1, 'time': 13.255234956741333}
response sent successfully
{'total': 11531, 'batches': 2, 'time': 22.22915029525757}
response sent successfully
{'total': 19943, 'batches': 3, 'time': 29.285478115081787}
response sent successfully
{'total': 27380, 'batches': 4, 'time': 35.01780915260315}
response sent successfully
{'total': 29906, 'batches': 5, 'time': 37.79424715042114}


{'total': 29906, 'batches': 5, 'time': 37.79424715042114}

In [15]:
from itertools import chain

all_refs = chain.from_iterable(list(cd['refs']))
all_ids = list(cd['id'])

print(len(set(all_ids).intersection(all_refs)))

9558


In [16]:
#test script to see if references is actually working or not
a = cd.loc[cd['id'] == '000-047-609-862-89X']
all_refs = list(a['refs'])[0]

b = cd.loc[cd['id'] == all_refs[8]]
b 

,category_list,id,mesh_term,refs,title
29543,"[Antigen, T-cell receptor, Immunology, Inflamm...",079-541-883-399-471,"[D019264, D000818, D000831, D015551, D002478, ...","[001-549-103-474-087, 004-852-459-962-744, 005...",A requisite role for induced regulatory T cell...


In [17]:
a = list(db[db['lens_id']=='023-065-198-031-445']['mesh_terms'])
print(a)



[[{'mesh_heading': 'Animals', 'mesh_id': 'D000818', 'qualifier_id': None, 'qualifier_name': None}, {'mesh_heading': 'Apoptosis', 'mesh_id': 'D017209', 'qualifier_id': 'Q000187', 'qualifier_name': 'drug effects'}, {'mesh_heading': 'B-Lymphocytes', 'mesh_id': 'D001402', 'qualifier_id': 'Q000187', 'qualifier_name': 'drug effects'}, {'mesh_heading': 'CD40 Ligand', 'mesh_id': 'D023201', 'qualifier_id': None, 'qualifier_name': None}, {'mesh_heading': 'Down-Regulation', 'mesh_id': 'D015536', 'qualifier_id': 'Q000187', 'qualifier_name': 'drug effects'}, {'mesh_heading': 'Genes, myc', 'mesh_id': 'D016259', 'qualifier_id': 'Q000276', 'qualifier_name': 'immunology'}, {'mesh_heading': 'Ligands', 'mesh_id': 'D008024', 'qualifier_id': None, 'qualifier_name': None}, {'mesh_heading': 'Membrane Glycoproteins', 'mesh_id': 'D008562', 'qualifier_id': 'Q000494', 'qualifier_name': 'pharmacology'}, {'mesh_heading': 'Mice', 'mesh_id': 'D051379', 'qualifier_id': None, 'qualifier_name': None}, {'mesh_heading': 